In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
igfold = 4
os.system('mkdir -p ./data_folds/fold_' + str(igfold))

0

In [2]:
!nvidia-smi

Fri Jan 13 11:51:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   40C    P0    49W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   

In [3]:
import cudf
import glob
import gc

from tqdm import tqdm

In [4]:
lb = False
path = '../data/'
type_weight = {0:1, 1:6, 2:3}
no_files = 5
path = '../../data/'
    
df_type = cudf.DataFrame({
    'type': ['clicks', 'carts', 'orders'],
    'type_': [0, 1, 2]
})

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

In [5]:
import pickle

sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
if igfold == 0:
    sess_eval = sessions[0]+sessions[1]
elif igfold == 1:
    sess_eval = sessions[2]+sessions[3]
elif igfold == 2:
    sess_eval = sessions[4]+sessions[5]
elif igfold == 3:
    sess_eval = sessions[6]+sessions[7]
elif igfold == 4:
    sess_eval = sessions[8]+sessions[9]
print(len(sess_eval))

356746


In [6]:
# !rm -rf ./data/
!mkdir -p ./data/

In [7]:
files = sorted(
    glob.glob(path + '/train/interim/*.parquet')
)
files_split = [glob.glob('../../data/test.parquet')] + [glob.glob('./data/xgb_train_x.parquet')] + list_in_chunks(files, no_chunks=len(files)//no_files)

In [8]:
len(files_split)

27

### 0) Clicks->Orders

In [9]:
out = []
for e, file in tqdm(enumerate(files_split)):
    df = cudf.read_parquet(file).reset_index(drop=True)
    df = df.merge(
        df_type,
        how='left',
        on='type'
    )
    if any([True for x in file if 'xgb_train_x.parquet' in x]):
        print(file)
        df = df[df['session'].isin(sess_eval)]
    else:
        df = df.loc[~(df['session'].isin(sess_eval))]

    df['session'] = df['session'].astype('int32')
    df['aid'] = df['aid'].astype('int32')
    df.ts = (df.ts/1000).astype('int32')
    df_tmp = df[df['type']!='clicks']
    df = df.merge(
        df_tmp,
        how='left',
        on=['session']
    )
    df = df[~df.aid_y.isna()]
    df = df[df['type_x']=='clicks']
    df = df.loc[((df.ts_x - df.ts_y).abs()< 7*24 * 60 * 60)]
    df = df[['session', 'aid_x', 'aid_y']].drop_duplicates()
    df = df.groupby(['aid_x', 'aid_y']).count().reset_index()
    out.append(df.to_pandas())
    del df, df_tmp
    gc.collect()

1it [00:02,  2.12s/it]

['./data/xgb_train_x.parquet']


27it [00:45,  1.70s/it]


In [10]:
df = cudf.concat([cudf.from_pandas(x) for x in out])
del out
gc.collect()
df = df.groupby(['aid_x','aid_y']).session.sum()
gc.collect()
df = df.reset_index()
df.columns = ['aid_x', 'aid_y', 'wgt']
df = df[df['wgt']>1]
df = df.sort_values(['aid_x','wgt'],ascending=[True,False])
df = df.reset_index(drop=True)
gc.collect()
df['n'] = df.groupby('aid_x').aid_y.cumcount()
#df = df.loc[df.n<15].drop('n',axis=1)
df.to_parquet('./data_folds/fold_' + str(igfold) + '/top_15_c2o_v3.parquet')

In [11]:
del df
gc.collect()

14

### Generate Candidates

In [12]:
!nvidia-smi

Fri Jan 13 11:52:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   44C    P0    71W / 163W |    690MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:0A:00.0 Off |                    0 |
| N/A   

In [13]:
import cudf
import numpy as np
import gc
import os

import glob

In [14]:
def get_candidates(df, co, prefix, treshhold=2, treshhold_topn=15):
    df = df.merge(
        co[co['n']<treshhold_topn][['aid', 'cand', 'wgt']],
        how='left',
        on='aid'
    )
    df['wgt2'] = df['wgt']*df['score']
    df['wgt_cl'] = (df['type']=='clicks').astype('int8')*df['wgt']
    df['wgt_ca'] = (df['type']=='carts').astype('int8')*df['wgt']
    df['wgt_or'] = (df['type']=='orders').astype('int8')*df['wgt']
    df.drop(['aid', 'ts', 'type'], axis=1, inplace=True)
    df = df.groupby(['session', 'cand']).agg({
        'dummy': 'sum',
        'rank': 'min',
        'wgt': 'sum',
        'wgt2': 'sum',
        'wgt_cl': 'sum',
        'wgt_ca': 'sum',
        'wgt_or': 'sum'
    }).reset_index()
    df.columns = [
        'session', 'cand', 
        prefix+'_num_' + str(treshhold_topn),
        prefix+'_rank_min_'+ str(treshhold_topn), 
        prefix+'_wgt_' + str(treshhold_topn),
        prefix+'_wgt_2_'+ str(treshhold_topn), 
        prefix+'_wgt_cl_'+ str(treshhold_topn), 
        prefix+'_wgt_ca_'+ str(treshhold_topn), 
        prefix+'_wgt_or_'+ str(treshhold_topn)
    ]
    return(df)

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

In [15]:
prefix = 'c2o'

In [16]:
for dataset in ['train', 'sub']:
    for treshhold_topn in [15]:
        print(dataset, treshhold_topn)
        os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn))
        co = cudf.read_parquet('./data_folds/fold_' + str(igfold) + '/top_15_c2o_v3.parquet')
        co.columns = ['aid', 'cand', 'wgt', 'n']
        if dataset == 'train':
            df = cudf.read_parquet('./data/xgb_train_x.parquet')
            df = df[df['session'].isin(sess_eval)]
        else:
            df = cudf.read_parquet('../../data/test.parquet')
        df = df.sort_values(['session', 'ts'], ascending=[True, False])
        df['dummy'] = 1
        df['rank'] = df.groupby(['session']).dummy.cumsum()
        df['score'] = 1/df['rank']
        session_lists = list_in_chunks(df['session'].drop_duplicates().to_pandas().values.tolist(), no_chunks=10)
        out = []
        for session_list in session_lists:
            df_tmp = df[df['session'].isin(session_list)]
            df_tmp = get_candidates(df_tmp, co, prefix, treshhold=2, treshhold_topn=treshhold_topn)
            out.append(df_tmp)
        df = cudf.concat(out)
        df['session'] = df['session'].astype('int32')
        df['cand'] = df['cand'].astype('int32')
        df[prefix + '_num_'+ str(treshhold_topn)] = df[prefix + '_num_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_rank_min_'+ str(treshhold_topn)] = df[prefix + '_rank_min_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_wgt_'+ str(treshhold_topn)] = df[prefix + '_wgt_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_wgt_2_'+ str(treshhold_topn)] = df[prefix + '_wgt_2_'+ str(treshhold_topn)].astype('float32')
        df[prefix + '_wgt_cl_'+ str(treshhold_topn)] = df[prefix + '_wgt_cl_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_wgt_ca_'+ str(treshhold_topn)] = df[prefix + '_wgt_ca_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_wgt_or_'+ str(treshhold_topn)] = df[prefix + '_wgt_or_'+ str(treshhold_topn)].astype('int16')
        df.to_parquet('./data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn) + '/cand.parquet')
        del df
        gc.collect()

train 15
sub 15
